In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch
import os
import pandas as pd

%matplotlib notebook

In [2]:
def remove_excess(all_data, subcarrier_indices=None):
    one = [32 - 21, 32 - 7, 32, 32 + 7, 32 + 21]
    two = np.s_[0:6]
    three = np.s_[-5::]
    t = np.delete(all_data, one, axis=-1)  # subcarrier 0, and pilots
    t = np.delete(t, two, axis=-1)  # starting zeros
    t = np.delete(t, three, axis=-1)  # ending zeros

    if subcarrier_indices is not None:
        s = np.delete(subcarrier_indices, one, axis=0)
        s = np.delete(s, two, axis=0)
        s = np.delete(s, three, axis=0)

        return t, s

    return t


def calibrate_phase(phases, subcarriers):
    calibrated = np.zeros_like(phases)

    for index, phase in enumerate(phases):
        t_p = np.zeros_like(phase)
        m = subcarriers
        diff = 0
        eta = np.pi
        t_p[0] = phase[0]
        for i in range(1, len(phase)):
            # diff = 0
            if (phase[i] - phase[i - 1]) > eta:  # absolute value?
                diff += 0.75
            elif (phase[i] - phase[i - 1]) < -eta:
                diff -= 0.75

            t_p[i] = phase[i] - diff * 2 * eta

        k = (t_p[-1] - t_p[0]) / (m[-1] - m[0])
        b = np.sum(t_p) / len(t_p)

        calibrated[index] = t_p - k * m - b

    return calibrated


def smooth_amp(arr, window):
    # like honestly I don't even know how this works, it just does. like what??

    w_half = (window // 2) + 1

    cs_vec = np.cumsum(np.insert(arr, 0, 0))
    tails = np.array([(np.average(arr[:(w_half + i)]), np.average(arr[-(w_half + i):])) for i in range(0, window // 2)])
    average = (cs_vec[window:] - cs_vec[:-window]) / window  # ????

    return np.average([np.r_[tails[:, 0], average, tails[::-1, 1]], arr], axis=0)


vectorized_smoothing = np.vectorize(smooth_amp, excluded=['window'], signature='(2)->(2)')


In [3]:
stamps = os.listdir('time_stamps')
frames = os.listdir('video_frames')
position = os.listdir('position_data')
stp = 'time_stamps/'
frp = 'video_frames/'
pdp = 'position_data/'
stamps = sorted(stamps, key=lambda x: int(x[11:][:-4]))
frames = sorted(frames, key=lambda x: int(x[12:][:-4]))
position = sorted(position, key=lambda x: int(x[13:][:-4]))

all_stamps = []
for path in stamps:
    all_stamps.append(np.load(stp + path)) 
all_stamps = np.r_[*all_stamps]

position_data = []
for i, path in enumerate(position):
    x = np.load(pdp + path)
    # position_data.append(x[:,:,:2]) # limbs
    position_data.append(np.c_[np.mean(x[:, :, 0], axis=1), np.mean(x[:, :, 1], axis=1)])
all_positions = torch.tensor(np.concatenate(position_data, axis=0, dtype=np.float32))

csi_data = pd.read_pickle('awesome_sauce_actually_saved.pkl')
tpa_dat = csi_data[['phases', 'amplitudes']]

matched_csi_data = []

for index, val in enumerate(all_stamps):
    matched_csi_data.append(tpa_dat.iloc[abs(csi_data['time'] - val).idxmin()].to_numpy())

# all_positions has all the position truth data
# matched_csi_data has all the csi data which matches with each timestamp


In [4]:
phases = np.array(matched_csi_data)[:, 0]
amps = np.array(matched_csi_data)[:, 1]
subcarriers = np.arange(-32, 32)

# Phases

re_phases = np.empty((np.shape(phases)[0], 64))
for index, phase in enumerate(phases):
    re_phases[index] = phase

re_phases, subcarriers = remove_excess(re_phases, subcarriers)
calibrated_phases = calibrate_phase(re_phases, subcarriers)

# calibrated_phases = (calibrated_phases - np.mean(calibrated_phases)) / np.std(calibrated_phases)


# Amplitudes

re_amps = np.empty((np.shape(amps)[0], 64))

for index, amp in enumerate(amps):
    re_amps[index] = amp

re_amps = remove_excess(re_amps)

# 5, 7, 9, 11, 13?? higher means more stability, but might lose features of the data...
dn_amps = vectorized_smoothing(re_amps, window=9)

dn_amps = (dn_amps - np.mean(dn_amps)) / np.std(dn_amps)

final_data = torch.tensor(np.array(list(zip(calibrated_phases, dn_amps)), dtype=np.float32))

In [5]:
from torch.optim import RMSprop
# Define your MyNN-model

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()    

        self.conv1 = nn.Conv1d(2, 12, 25)
        self.conv2 = nn.Conv1d(12, 24, 13)
        self.conv3 = nn.Conv1d(24, 24, 7)

        self.avgpool = nn.AvgPool1d(6)

        self.fc1 = nn.Linear(24, 2)


    def __call__(self, x):
        """ The model's forward pass functionality.
        
        Parameters
        ----------
        x : Union[numpy.ndarray, mygrad.Tensor], shape=(N, T)
            The batch of size-N.
            
        Returns
        -------
        mygrad.Tensor, shape=(N, 2)
            The model's predictions for each of the N pieces of data in the batch.
        """
        # CHANGE DOCSTRING
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.avgpool(self.conv3(x))) # remove activation function?
        x = self.fc1(torch.squeeze(x)) # squeeze to remove dimension 1 at end, becomes 100x24 array
        return x
        
        

In [6]:
model = Model()
optimizer = RMSprop(model.parameters(), lr=1e-4)

In [7]:
from noggin import create_plot
plotter, fig, ax = create_plot(metrics=["loss"])

<IPython.core.display.Javascript object>

In [8]:
# Seperate Training set from Test set

random_gen = np.random.default_rng(0)

shuffle_idx = np.arange(len(final_data))
random_gen.shuffle(shuffle_idx)
shuffle_idx

array([1976, 6360, 8406, ..., 4411, 5921,  607])

In [9]:
sh_csi = final_data[shuffle_idx]
sh_position = all_positions[shuffle_idx]

training_input = sh_csi[:int(np.round(len(final_data)*0.8, 0))]
training_loss = sh_position[:int(np.round(len(final_data)*0.8, 0))]

testing_input = sh_csi[-int(np.round(len(final_data)*0.2, 0)):]
testing_loss = sh_position[-int(np.round(len(final_data)*0.2, 0)):]

In [10]:
def euclid_loss(pred, true): # pred (100, 2(x,y)) | true (100, 2(x,y))
    return torch.mean(torch.sqrt(torch.square(pred[:, 0] - true[:, 0]) + torch.square(pred[:, 1] - true[:, 1])))

In [11]:
batch_size = 300
num_epochs = 500


for epoch_cnt in range(num_epochs):
    epoch_cnt
    idxs = np.arange(len(training_input))
    np.random.shuffle(idxs)
    
    model.train()
    
    for batch_cnt in range(0, len(training_input) // batch_size):
        # random batch of our training data
        batch_indices = idxs[batch_cnt * batch_size : (batch_cnt + 1) * batch_size]

        batch = training_input[batch_indices]
        truth = training_loss[batch_indices]

        # perform the forward pass on our batch
        predictions = model(batch)
        
        # print(predictions.shape)
        # print(truth.shape)
        
        
        # calculate the loss
        loss = euclid_loss(predictions, truth)
        
        
        
        # perform backpropagation
        loss.backward()
        
        # update your parameters
        optimizer.step()
        optimizer.zero_grad()
        
        
        # plotter.set_train_batch({"loss" : loss.item(), "accuracy" : acc}, batch_size=batch_size)
        plotter.set_train_batch({"loss" : loss.item()}, batch_size=batch_size)

    
    # for batch_cnt in range(0, len(testing_input) // batch_size):
    #     idxs = np.arange(len(testing_input))
    #     batch_indices = idxs[batch_cnt * batch_size : (batch_cnt + 1) * batch_size]
    #     batch = test_data[batch_indices]
    #     truth = test_labels[batch_indices]
    #     
    #     with mg.no_autodiff:
    #         prediction = model(batch)
    #         acc = accuracy(prediction, truth)
        
        # plotter.set_test_batch({"accuracy" : acc}, batch_size=batch_size)
    
    plotter.set_train_epoch()
    # plotter.set_test_epoch()

KeyboardInterrupt: 

In [35]:
fig, ax = plt.subplots()

ax.set_ylim(0, 1080)
ax.set_xlim(0, 1920)

for index, csi in enumerate(testing_input[10:20]):
    model.eval()
    
    prediction = model(csi)
    
    prediction = prediction.detach().numpy()
    
    ax.plot()
    
    # ax.plot(*np.c_[testing_loss[index], prediction], 'x-')
    # ax.scatter(*testing_loss[index], s=100.)

<IPython.core.display.Javascript object>